# Save fastf1 data a to SQL database

Useful for generating quick reports during the race.


In [2]:
import fastf1
from fastf1.core import Session

GRAND_PRIX = 'Spain'
YEAR = 2024
SESSION = 'FP1'

session = fastf1.get_session(YEAR, GRAND_PRIX, SESSION)
session.load(messages=False)

req         WARNING 	DEFAULT CACHE ENABLED! (1.04 GB) /Users/diegomaradona/Library/Caches/fastf1
core           INFO 	Loading data for Spanish Grand Prix - Practice 1 [v3.3.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	No lap data for driver 27
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 27)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 21 drivers: ['1', '10', '11', '14', '16', '18', '2', '20'

In [3]:
session.weather_data

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:00:25.642000,27.5,37.0,1004.3,False,46.3,93,0.0
1,0 days 00:01:25.662000,27.5,36.0,1004.5,False,46.4,0,0.0
2,0 days 00:02:25.653000,27.5,36.0,1004.5,False,47.3,241,2.3
3,0 days 00:03:25.660000,27.6,34.0,1004.4,False,47.5,226,2.0
4,0 days 00:04:25.667000,27.7,35.0,1004.4,False,46.4,242,2.0
...,...,...,...,...,...,...,...,...
76,0 days 01:16:25.846000,28.3,37.0,1004.1,False,48.0,107,0.7
77,0 days 01:17:25.835000,28.4,36.0,1004.2,False,47.8,276,1.0
78,0 days 01:18:25.832000,28.5,36.0,1004.2,False,46.1,230,2.1
79,0 days 01:19:25.839000,28.5,37.0,1004.2,False,46.2,259,1.7


In [4]:
session.event

RoundNumber                                                   10
Country                                                    Spain
Location                                               Barcelona
OfficialEventName    FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2024
EventDate                                    2024-06-23 00:00:00
EventName                                     Spanish Grand Prix
EventFormat                                         conventional
Session1                                              Practice 1
Session1Date                           2024-06-21 13:30:00+02:00
Session1DateUtc                              2024-06-21 11:30:00
Session2                                              Practice 2
Session2Date                           2024-06-21 17:00:00+02:00
Session2DateUtc                              2024-06-21 15:00:00
Session3                                              Practice 3
Session3Date                           2024-06-22 12:30:00+02:00
Session3DateUtc          

In [27]:
from pandas import DataFrame
from sqlalchemy import create_engine
from typing import cast


class Formula1Databases:
    """
    Quickly fetch, extract features and save `fastf1` data to a SQL database
    """

    def __init__(self, session: Session) -> None:
        self.laps = session.laps.copy()
        self.weather_data = cast(DataFrame, session.weather_data).copy()
        self.telemetry = session.car_data.copy()
        self.event = session.event.copy()
        self._session_start_date = session.session_info['StartDate']

        self.sql_engine = None
        self.df_laps: DataFrame | None = None
        self.df_weather: DataFrame | None = None

        self._rename_laps_columns()
        self._rename_weather_columns()

    def get_weather_df(self) -> DataFrame:
        df = self.weather_data.copy()
        df = self._calc_timedelta_from_session_start(df, ['time'])

        self.df_weather = df

        return df

    def save_to_SQL_database(self, database_name: str):
        """
        @param database_name str must contain `.db` at the end
        """
        if not self.sql_engine:
            self.sql_engine = create_engine(f'sqlite:///{database_name}')

        if self.df_laps is not None:
            self.df_laps.to_sql(name="Laps", con=self.sql_engine)
            print(f'> table "Laps" saved to {database_name}')

        if self.df_weather is not None:
            self.df_weather.to_sql(name="Weather", con=self.sql_engine)
            print(f'> table "Weather" saved to {database_name}')

    def get_laps_df(self) -> DataFrame:
        """Return the `laps` data related to the given session as a pandas DataFrame"""
        df = self.laps.copy()
        df = self._convert_timedelta_to_seconds(df, ['lap_time',
                                                     'sector_1_time', 'sector_2_time', 'sector_3_time'])
        df = self._calc_timedelta_from_session_start(
            df, ['pit_out', 'pit_in', 'lap_end'])
        unused_columns = ['DeletedReason', 'FastF1Generated', 'IsAccurate', 'LapStartTime',
                          'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime']
        df.drop(columns=[col for col in unused_columns if col in df.columns],
                inplace=True,
                axis=1)

        self.df_laps = df

        return df

    def _rename_weather_columns(self) -> None:
        weather_new_column_names = {
            "Time": "time",
            "AirTemp": "air_temperature_in_C",
            "Humidity": "relative_humidity_in_percents",
            "Pressure": "air_pressure_in_mbar",
            "Rainfall": "is_raining",
            "TrackTemp": "track_temperature_in_C",
            "WindDirection": "wind_direction_in_grads",
            "WindSpeed": "wind_speed_in_ms"
        }
        self.weather_data.rename(
            columns=weather_new_column_names, inplace=True)

    def _rename_laps_columns(self) -> None:
        laps_new_column_names = {
            'Driver': 'driver_name',
            'DriverNumber': 'driver_number',
            'LapNumber': 'lap_number',
            'Stint': 'stint',
            'SpeedI1': 'sector_1_speed_trap_in_km',
            'SpeedI2': 'sector_2_speed_trap_in_km',
            'SpeedFL': 'finish_line_speed_trap_in_km',
            'SpeedST': 'longest_strait_speed_trap_in_km',
            'IsPersonalBest': 'is_personal_best',
            'Compound': 'tyre_compound',
            'TyreLife': 'tyre_life_in_laps',
            'FreshTyre': 'is_fresh_tyre',
            'Position': 'position',
            'LapTime': 'lap_time',
            'Sector1Time': 'sector_1_time',
            'Sector2Time': 'sector_2_time',
            'Sector3Time': 'sector_3_time',
            'LapStartDate': 'lap_start_datetime',
            'Time': 'lap_end',
            'PitInTime': 'pit_in',
            'PitOutTime': 'pit_out',
            'Team': 'team_name',
            'TrackStatus': 'track_status',
            'Deleted': 'is_lap_deleted'
        }
        self.laps.rename(columns=laps_new_column_names, inplace=True)

    def _convert_timedelta_to_seconds(self, old_df: DataFrame, columns: list[str]) -> DataFrame:
        df = old_df.copy()

        for col in columns:
            if not col in df.columns:
                continue
            df[col + '_in_seconds'] = df[col].dt.total_seconds()
            df.drop(columns=[col], inplace=True, axis=1)

        return df

    def _calc_timedelta_from_session_start(self, old_df: DataFrame, columns: list[str]) -> DataFrame:
        df = old_df.copy()

        for col in columns:
            if not col in df.columns:
                continue
            df[col + '_datetime'] = self._session_start_date + df[col]
            df.drop(columns=[col], inplace=True, axis=1)

        return df

In [28]:
spanish_session = Formula1Databases(session=session)

In [22]:

spanish_session.get_laps_df()

,driver_name,driver_number,lap_number,stint,sector_1_speed_trap_in_km,sector_2_speed_trap_in_km,finish_line_speed_trap_in_km,longest_strait_speed_trap_in_km,is_personal_best,tyre_compound,...,track_status,position,is_lap_deleted,lap_time_in_seconds,sector_1_time_in_seconds,sector_2_time_in_seconds,sector_3_time_in_seconds,pit_out_datetime,pit_in_datetime,lap_end_datetime
0,VER,1,1.0,1.0,263.0,213.0,282.0,225.0,False,HARD,...,1,NaN,None,NaN,NaN,35.704,26.353,2024-06-21 13:47:35.427,NaT,2024-06-21 13:49:02.676
1,VER,1,2.0,1.0,283.0,287.0,281.0,315.0,True,HARD,...,1,NaN,None,76.254,22.500,30.770,22.984,NaT,NaT,2024-06-21 13:50:18.930
2,VER,1,3.0,1.0,183.0,174.0,NaN,148.0,False,HARD,...,1,NaN,None,113.995,37.089,43.413,33.493,NaT,2024-06-21 13:52:10.252,2024-06-21 13:52:12.925
3,VER,1,4.0,2.0,87.0,143.0,286.0,223.0,False,HARD,...,1,NaN,None,129.370,53.819,50.332,25.219,2024-06-21 13:52:33.412,NaT,2024-06-21 13:54:22.295
4,VER,1,5.0,2.0,288.0,303.0,284.0,317.0,True,HARD,...,1,NaN,None,75.424,22.229,30.458,22.737,NaT,NaT,2024-06-21 13:55:37.719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,PIA,81,25.0,6.0,272.0,262.0,NaN,304.0,False,MEDIUM,...,1,NaN,None,83.305,23.511,31.976,27.818,NaT,2024-06-21 14:41:44.036,2024-06-21 14:41:46.718
536,PIA,81,26.0,7.0,268.0,258.0,282.0,250.0,False,SOFT,...,1,NaN,None,103.611,47.185,32.579,23.847,2024-06-21 14:42:11.042,NaT,2024-06-21 14:43:30.329
537,PIA,81,27.0,7.0,272.0,253.0,282.0,313.0,False,SOFT,...,1,NaN,None,79.723,23.062,32.377,24.284,NaT,NaT,2024-06-21 14:44:50.052
538,PIA,81,28.0,7.0,207.0,177.0,40.0,263.0,False,SOFT,...,1,NaN,None,109.685,28.978,38.690,42.017,NaT,NaT,2024-06-21 14:46:39.737


In [17]:
spanish_session.save_to_SQL_database("Spanish_Session")

In [29]:
spanish_session.get_weather_df()

,air_temperature_in_C,relative_humidity_in_percents,air_pressure_in_mbar,is_raining,track_temperature_in_C,wind_direction_in_grads,wind_speed_in_ms,time_datetime
0,27.5,37.0,1004.3,False,46.3,93,0.0,2024-06-21 13:30:25.642
1,27.5,36.0,1004.5,False,46.4,0,0.0,2024-06-21 13:31:25.662
2,27.5,36.0,1004.5,False,47.3,241,2.3,2024-06-21 13:32:25.653
3,27.6,34.0,1004.4,False,47.5,226,2.0,2024-06-21 13:33:25.660
4,27.7,35.0,1004.4,False,46.4,242,2.0,2024-06-21 13:34:25.667
...,...,...,...,...,...,...,...,...
76,28.3,37.0,1004.1,False,48.0,107,0.7,2024-06-21 14:46:25.846
77,28.4,36.0,1004.2,False,47.8,276,1.0,2024-06-21 14:47:25.835
78,28.5,36.0,1004.2,False,46.1,230,2.1,2024-06-21 14:48:25.832
79,28.5,37.0,1004.2,False,46.2,259,1.7,2024-06-21 14:49:25.839
